# Project Overview
Perform data analysis and data preparation tasks to train a Linear Regression model to predict future ROI (Return On Investment) of variable ad spend budgets across multiple channels including search, video, social media, and email using Snowpark for Python, Snowpark ML and Streamlit. By the end of the session, you will have an interactive web application deployed visualizing the ROI of different allocated advertising spend budgets.

### Data Engineering -- Data Analysis and Data Preparation
In this Notebook, we will focus on Data Engineering in Snowflake using Snowpark for Python.

* Establish secure connection to Snowflake
* Load data from Snowflake tables into Snowpark DataFrames
* Perform Exploratory Data Analysis on Snowpark DataFrames
* Pivot and Join data from multiple tables using Snowpark DataFrames
* Demostrate how to automate data preparation using Snowflake Tasks

*For environment setup including loading data into Snowflake tables, and step-by-step instructions, please refer to the [QuickStart Guide](https://quickstarts.snowflake.com/guide/getting_started_with_dataengineering_ml_using_snowpark_python/index.html#0).*

### Import Libraries

In [1]:
# Snowpark for Python
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import month,year,col,sum
from snowflake.snowpark.version import VERSION

# Misc
import json
import logging 
logger = logging.getLogger("snowflake.snowpark.session")
logger.setLevel(logging.ERROR)

In [ ]:
# Create Snowflake Session object
connection_parameters = json.load(open('connection.json'))
print(connection_parameters)

sessionBuilder = Session.builder
for key, value in connection_parameters.items():
    sessionBuilder.config(key,value)

session = sessionBuilder.create()
session.sql_simplifier_enabled = True

snowflake_environment = session.sql('select current_user(), current_version()').collect()
snowpark_version = VERSION

# Current Environment Details
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))

### Establish Secure Connection to Snowflake

Using the Snowpark Python API, it’s quick and easy to establish a secure connection between Snowflake and Notebook.

 *Note: Other connection options include Username/Password, MFA, OAuth, Okta, SSO*

TIP: Learn more about [Session](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/session) object.

In [2]:
# Create Snowflake Session object
connection_parameters = json.load(open('connection.json'))
print(connection_parameters)
session = Session.builder.configs(connection_parameters).create()
session.sql_simplifier_enabled = True

snowflake_environment = session.sql('select current_user(), current_version()').collect()
snowpark_version = VERSION

# Current Environment Details
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))

{'account': 'usdinjc-zcb36955', 'user': 'GGARCIAG', 'password': 'Red$heep97', 'role': 'ACCOUNTADMIN', 'warehouse': 'DASH_L', 'database': 'DASH_DB', 'schema': 'DASH_SCHEMA'}


DatabaseError: 250001 (08001): Failed to connect to DB: usdinjc-zcb36955.snowflakecomputing.com:443. Your free trial has ended and all of your virtual warehouses have been suspended. Add billing information in the Snowflake web UI to continue using the full set of Snowflake features.

### Load Aggregated Campaign Spend Data from Snowflake table into Snowpark DataFrame

Let's first load the campaign spend data. This table contains ad click data that has been aggregated to show daily spend across digital ad channels including search engines, social media, email and video.

*Note: Some other ways to load data in a Snowpark DataFrame*
* *session.sql("select col1, col2... from tableName")*
* *session.read.options({"field_delimiter": ",", "skip_header": 1}).schema(user_schema).csv("@mystage/testCSV.csv")*
* *session.read.parquet("@stageName/path/to/file")*
* *session.create_dataframe([1,2,3], schema=["col1"])*

TIP: Learn more about [Snowpark DataFrames](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/dataframe).


In [ ]:
snow_df_spend = session.table('campaign_spend')
snow_df_spend.queries

Actions like *show(), collect(), count()* send the DataFrame SQL for execution on the server

*Note: History object provides the query ID which can be helpful for debugging as well as the SQL query executed on the server.*

In [ ]:
with session.query_history() as history:
    snow_df_spend.show(20)
history.queries

### Total Spend per Year and Month For All Channels

Let's transform the data so we can see total cost per year/month per channel using _group_by()_ and _agg()_ Snowpark DataFrame functions.

TIP: For a full list of functions, refer to the [documentation](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/functions).

In [ ]:
# Stats per Month per Channel
snow_df_spend_per_channel = snow_df_spend.group_by(year('DATE'), month('DATE'),'CHANNEL').agg(sum('TOTAL_COST').as_('TOTAL_COST')).\
    with_column_renamed('"YEAR(DATE)"',"YEAR").with_column_renamed('"MONTH(DATE)"',"MONTH").sort('YEAR','MONTH')

snow_df_spend_per_channel.show(10)

### Pivot on Channel: Total Spend Across All Channels

 Let's further transform the campaign spend data so that **each row will represent total cost across all channels** per year/month using _pivot()_ and _sum()_ Snowpark DataFrame functions. This transformation will enable us to join with the revenue table such that we will have our input features and target variable in a single table for model training. 

 TIP: For a full list of functions, refer to the [documentation](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/functions).

In [ ]:
snow_df_spend_per_month = snow_df_spend_per_channel.pivot('CHANNEL',['search_engine','social_media','video','email']).sum('TOTAL_COST').sort('YEAR','MONTH')
snow_df_spend_per_month = snow_df_spend_per_month.select(
    col("YEAR"),
    col("MONTH"),
    col("'search_engine'").as_("SEARCH_ENGINE"),
    col("'social_media'").as_("SOCIAL_MEDIA"),
    col("'video'").as_("VIDEO"),
    col("'email'").as_("EMAIL")
)
snow_df_spend_per_month.show()

### Save Transformed Data into Snowflake Table

Let's save the transformed data into a Snowflake table *SPEND_PER_MONTH*.

In [ ]:
snow_df_spend_per_month.write.mode('overwrite').save_as_table('SPEND_PER_MONTH')

### Automation: Run Campaign Spend Data Transformations As a Snowflake Task

*Note: Optionally you can run all these transformations as an automated task by deploying the code to Snowflake as a Snowpark Stored Procedure and executing it as a Snowflake Task.*

TIP: Learn more about [Stored Procedures](https://docs.snowflake.com/en/sql-reference/stored-procedures-python) and [Snowflake Tasks](https://docs.snowflake.com/en/sql-reference/sql/create-task).

In [ ]:
def campaign_spend_data_pipeline(session: Session) -> str:
  # DATA TRANSFORMATIONS
  # Perform the following actions to transform the data

  # Load the campaign spend data
  snow_df_spend_t = session.table('campaign_spend')

  # Transform the data so we can see total cost per year/month per channel using group_by() and agg() Snowpark DataFrame functions
  snow_df_spend_per_channel_t = snow_df_spend_t.group_by(year('DATE'), month('DATE'),'CHANNEL').agg(sum('TOTAL_COST').as_('TOTAL_COST')).\
      with_column_renamed('"YEAR(DATE)"',"YEAR").with_column_renamed('"MONTH(DATE)"',"MONTH").sort('YEAR','MONTH')

  # Transform the data so that each row will represent total cost across all channels per year/month using pivot() and sum() Snowpark DataFrame functions
  snow_df_spend_per_month_t = snow_df_spend_per_channel_t.pivot('CHANNEL',['search_engine','social_media','video','email']).sum('TOTAL_COST').sort('YEAR','MONTH')
  snow_df_spend_per_month_t = snow_df_spend_per_month_t.select(
      col("YEAR"),
      col("MONTH"),
      col("'search_engine'").as_("SEARCH_ENGINE"),
      col("'social_media'").as_("SOCIAL_MEDIA"),
      col("'video'").as_("VIDEO"),
      col("'email'").as_("EMAIL")
  )

  # Save transformed data
  snow_df_spend_per_month_t.write.mode('overwrite').save_as_table('SPEND_PER_MONTH')

# Register data pipelining function as a Stored Procedure so it can be run as a task
session.sproc.register(
  func=campaign_spend_data_pipeline,
  name="campaign_spend_data_pipeline",
  packages=['snowflake-snowpark-python'],
  is_permanent=True,
  stage_location="@dash_sprocs",
  replace=True)

campaign_spend_data_pipeline_task = """
CREATE OR REPLACE TASK campaign_spend_data_pipeline_task
    WAREHOUSE = 'DASH_L'
    SCHEDULE  = '3 MINUTE'
AS
    CALL campaign_spend_data_pipeline()
"""
session.sql(campaign_spend_data_pipeline_task).collect()
    

### Total Revenue per Year And Month

Now let's load revenue table and transform the data into revenue per year/month using *group_by() and agg()* functions.

In [ ]:
snow_df_revenue = session.table('monthly_revenue')
snow_df_revenue_per_month = snow_df_revenue.group_by('YEAR','MONTH').agg(sum('REVENUE')).sort('YEAR','MONTH').with_column_renamed('SUM(REVENUE)','REVENUE')
snow_df_revenue_per_month.show()

### Join Total Spend and Total Revenue per Year and Month Across All Channels

Next let's **join this revenue data with the transformed campaign spend data** so that our input features (i.e. cost per channel) and target variable (i.e. revenue) can be loaded into a single table for model training. 

In [ ]:
snow_df_spend_and_revenue_per_month = snow_df_spend_per_month.join(snow_df_revenue_per_month, ["YEAR","MONTH"])
snow_df_spend_and_revenue_per_month.show()

### >>>>>>>>>> *Examine Snowpark DataFrame Query and Execution Plan* <<<<<<<<<<

Snowpark makes is really convenient to look at the DataFrame query and execution plan using _explain()_ Snowpark DataFrame function.

In [ ]:
snow_df_spend_and_revenue_per_month.explain()

### Save Transformed Data into Snowflake Table

Let's save the transformed data into a Snowflake table *SPEND_AND_REVENUE_PER_MONTH*.

In [ ]:
snow_df_spend_and_revenue_per_month.write.mode('overwrite').save_as_table('SPEND_AND_REVENUE_PER_MONTH')

### Automation: Run Monthly Revenue Data Transformations As a Snowflake Task

*Note: Optionally you can run all these transformations as an automated task by deploying the code to Snowflake as a Snowpark Stored Procedure and executing it as a Snowflake Task. In this task, notice the AFTER campaign_spend_data_pipeline_task clause which makes it a dependant task.*

TIP: Learn more about [Stored Procedures](https://docs.snowflake.com/en/sql-reference/stored-procedures-python) and [Snowflake Tasks](https://docs.snowflake.com/en/sql-reference/sql/create-task).

In [ ]:
def monthly_revenue_data_pipeline(session: Session) -> str:
  # Load revenue table and transform the data into revenue per year/month using group_by and agg() functions
  snow_df_spend_per_month_t = session.table('spend_per_month')
  snow_df_revenue_t = session.table('monthly_revenue')
  snow_df_revenue_per_month_t = snow_df_revenue_t.group_by('YEAR','MONTH').agg(sum('REVENUE')).sort('YEAR','MONTH').with_column_renamed('SUM(REVENUE)','REVENUE')

  # Join revenue data with the transformed campaign spend data so that our input features (i.e. cost per channel) and target variable (i.e. revenue) can be loaded into a single table for model training
  snow_df_spend_and_revenue_per_month_t = snow_df_spend_per_month_t.join(snow_df_revenue_per_month_t, ["YEAR","MONTH"])

  # SAVE in a new table for the next task
  snow_df_spend_and_revenue_per_month_t.write.mode('overwrite').save_as_table('SPEND_AND_REVENUE_PER_MONTH')

# Register data pipelining function as a Stored Procedure so it can be run as a task
session.sproc.register(
  func=monthly_revenue_data_pipeline,
  name="monthly_revenue_data_pipeline",
  packages=['snowflake-snowpark-python'],
  is_permanent=True,
  stage_location="@dash_sprocs",
  replace=True)

monthly_revenue_data_pipeline_task = """
  CREATE OR REPLACE TASK monthly_revenue_data_pipeline_task
      WAREHOUSE = 'DASH_L'
      AFTER campaign_spend_data_pipeline_task
  AS
      CALL monthly_revenue_data_pipeline()
  """
session.sql(monthly_revenue_data_pipeline_task).collect()

### Resume Tasks

*Note: Snowflake Tasks are suspended by default so you need to resume them by executing the following commands.*

In [ ]:
# session.sql("alter task monthly_revenue_data_pipeline_task resume").collect()
# session.sql("alter task campaign_spend_data_pipeline_task resume").collect()

### Suspend Tasks

*Note: For the sake of this lab, if you resume the above tasks, suspend them to avoid unecessary resource utilization by executing the following commands.*

In [ ]:
# session.sql("alter task campaign_spend_data_pipeline_task suspend").collect()
# session.sql("alter task monthly_revenue_data_pipeline_task suspend").collect()

_For comments and feedback, please reach out to dash.desai@snowflake.com | Follow on [Twitter](https://twitter.com/iamontheinet)_ 